In [28]:
import xarray as xr 
import rioxarray as rio
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#### Load & Read Files

In [29]:
# CURRENTLY LOADING...
# EPICA - 5km spatial resolution - 100 year temporal resolution
file = "/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/input_nc_files/ciscyc.5km.epica.ex.100a.nc"

# Open file
grip_10km_1ka = xr.open_dataset(file)

### View NetCDF File

Dimensions, Coordinates, Data Variables, & Attributes

In [30]:
# Print file information
    # Better readability when you do NOT use a print() statement
grip_10km_1ka

<xarray.Dataset> Size: 7GB
Dimensions:           (age: 1200, y: 601, x: 301)
Coordinates:
    lon               (y, x) float32 724kB ...
    lat               (y, x) float32 724kB ...
    time              (age) object 10kB ...
  * x                 (x) float64 2kB -2.5e+06 -2.495e+06 ... -1.005e+06 -1e+06
  * y                 (y) float64 5kB 0.0 5e+03 1e+04 ... 2.995e+06 3e+06
  * age               (age) float64 10kB 119.9 119.8 119.7 ... 0.2 0.1 -0.0
Data variables:
    tempicethk_basal  (age, y, x) float32 868MB ...
    temppabase        (age, y, x) float32 868MB ...
    thk               (age, y, x) float32 868MB ...
    uvelbase          (age, y, x) float32 868MB ...
    uvelsurf          (age, y, x) float32 868MB ...
    vvelbase          (age, y, x) float32 868MB ...
    vvelsurf          (age, y, x) float32 868MB ...
    mapping           float64 8B ...
    pism_config       int8 1B ...
    topg              (age, y, x) float32 868MB ...
Attributes:
    Conventions:  CF-1.5
    command:      julien@polaris 2023-04-05 09:43:36 UTC: ./postprocess_ciscy...
    history:      julien@polaris 2023-04-05 09:43:36 UTC: ./postprocess_ciscy...
    proj4:        +units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95...
    source:       PISM stable v0.7.1-14-gb9fc6b0
    author:       Julien Seguinot
    title:        Cordilleran ice sheet glacial cycle simulations spatial dia...
    subtitle:     5km EPICA simulation
    institution:  Stockholm University, Sweden and ETH Zürich, Switzerland

#### Extracting Individual Variables

In [31]:
time = grip_10km_1ka['time']
thk = grip_10km_1ka['thk'] # ice thickness
bed = grip_10km_1ka['topg'] # bedrock surface elevation

uvel_base = grip_10km_1ka['uvelbase'] # x-component of the horizontal velocity of ice at the base of ice
vvel_base = grip_10km_1ka['vvelbase'] # y-component of the horizontal velocity of ice at the base of ice

uvel_surf = grip_10km_1ka['uvelsurf'] # x-component of the horizontal velocity of ice at ice surface
vvel_surf = grip_10km_1ka['vvelsurf'] # y-component of the horizontal velocity of ice at ice surface

#### Setting CRS

****You should not need to change the CRS information here between files because it should be the same for all the files from this database 

In [32]:
# ice thickness
thk = thk.rio.set_spatial_dims(x_dim='x', y_dim='y')
thk.rio.crs
thk.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# bedrock surface elevation
bed = bed.rio.set_spatial_dims(x_dim='x', y_dim='y')
bed.rio.crs
bed.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# x-component of the horizontal velocity of ice at the base of ice
uvel_base = uvel_base.rio.set_spatial_dims(x_dim='x', y_dim='y')
uvel_base.rio.crs
uvel_base.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# y-component of the horizontal velocity of ice at the base of ice
vvel_base = vvel_base.rio.set_spatial_dims(x_dim='x', y_dim='y')
vvel_base.rio.crs
vvel_base.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# x-component of the horizontal velocity of ice at ice surface
uvel_surf = uvel_surf.rio.set_spatial_dims(x_dim='x', y_dim='y')
uvel_surf.rio.crs
uvel_surf.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# y-component of the horizontal velocity of ice at ice surface
vvel_surf = vvel_surf.rio.set_spatial_dims(x_dim='x', y_dim='y')
vvel_surf.rio.crs
vvel_surf.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)


<xarray.DataArray 'vvelsurf' (age: 1200, y: 601, x: 301)> Size: 868MB
[217081200 values with dtype=float32]
Coordinates:
    lon      (y, x) float32 724kB ...
    lat      (y, x) float32 724kB ...
    time     (age) object 10kB ...
  * x        (x) float64 2kB -2.5e+06 -2.495e+06 -2.49e+06 ... -1.005e+06 -1e+06
  * y        (y) float64 5kB 0.0 5e+03 1e+04 ... 2.99e+06 2.995e+06 3e+06
  * age      (age) float64 10kB 119.9 119.8 119.7 119.6 ... 0.3 0.2 0.1 -0.0
    mapping  int64 8B 0
Attributes:
    units:        m year-1
    valid_range:  [-1000000.  1000000.]
    long_name:    y-component of the horizontal velocity of ice at ice surface
    pism_intent:  diagnostic

## Save to GeoTIFF File

In [33]:
#USE THIS SO YOU DONT WRITE OVER PREEXISTING FILES
file_name_addition = "_5km_epica_1ka"

In [34]:
# ice thickness
thk.rio.to_raster(f"ice_thickness{file_name_addition}.tif")

# bedrock surface elevation
bed.rio.to_raster(f"bed_topo{file_name_addition}.tif")

# x-component of the horizontal velocity of ice at the base of ice
uvel_base.rio.to_raster(f"x_basal_velocity{file_name_addition}.tif")

# y-component of the horizontal velocity of ice at the base of ice
vvel_base.rio.to_raster(f"y_basal_velocity{file_name_addition}.tif")

# x-component of the horizontal velocity of ice at ice surface
uvel_surf.rio.to_raster(f"x_surf_velocity{file_name_addition}.tif")

# y-component of the horizontal velocity of ice at ice surface
vvel_surf.rio.to_raster(f"y_surf_velocity{file_name_addition}.tif")

In [35]:
print(uvel_base.rio.crs)


EPSG:3978


In [36]:
tif = "/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/ice_thickness.tif"

In [37]:
try:
    with rasterio.open("/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/ice_thickness.tif") as src:
        image_array = src.read()  # Read all bands as a NumPy array
        print(f"Image array shape: {image_array.shape}")
        print(f"Coordinate Reference System (CRS): {src.crs}")
        print(f"Transform: {src.transform}")
except FileNotFoundError:
    print("Error: The specified GeoTIFF file was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Image array shape: (120, 601, 301)
Coordinate Reference System (CRS): EPSG:3978
Transform: | 5000.00, 0.00,-2502500.00|
| 0.00, 5000.00,-2500.00|
| 0.00, 0.00, 1.00|


### Visualize Data

In [38]:
# data_crs = ccrs.epsg(3978)
# fig,ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': data_crs})

# bounds = bed.rio.bounds()
# ax.set_extent([bounds[0], bounds[2], bounds[1], bounds[3]], crs=data_crs)

# ax.coastlines(resolution='10m')
# ax.add_feature(cfeature.BORDERS)
# ax.add_feature(cfeature.LAND, facecolor='lightgray')
# ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
# ax.gridlines(draw_labels=True)

# first_frame = bed.isel(age=0).values
# img = ax.imshow(first_frame, cmap='viridis', origin='lower',
#                 extent=bounds, transform=data_crs)
# cbar = fig.colorbar(img, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
# cbar.set_label('Ground ELevation (meters)') 

# def update(i):
#     data = bed.isel(age=i)
#     img.set_data(data)
#     # title.set_text(f"Time: {data['age'].values}")
#     return [img]

# ani = animation.FuncAnimation(fig, update, frames=bed.sizes["age"], interval = 200, blit = False)
# # HTML(ani.to_jshtml())
# ani.save('bed_grip.gif', writer='Pillow', fps=20)